## Install Packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from pytorch_nndct.apis import torch_quantizer, dump_xmodel
print("PyTorch version is ", torch.__version__)
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import h5py as h5
from sklearn.metrics import classification_report, confusion_matrix
import random
import time
from random import shuffle
import glob
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
from progressbar import ProgressBar
import matplotlib.pyplot as plt

!pip3 install torchsummary 
from torchsummary import summary

## Select GPU device

In [ ]:
# use GPU if available   
if (torch.cuda.device_count() > 0):
    print('You have',torch.cuda.device_count(),'CUDA devices available')
    for i in range(torch.cuda.device_count()):
        print(' Device',str(i),': ',torch.cuda.get_device_name(i))
    print('Selecting device 0..')
    device = torch.device('cuda:0')
else:
    print('No CUDA devices available..selecting CPU')
    device = torch.device('cpu')

## Download the 2018.01 Dataset from Deepsig
Deepsig has released multiple data sets that can be found here https://www.deepsig.io/datasets

This data set contains RF data with 24 different modulations at various SNR. Each of the 2555904 data inputs is 1024 samples long of complex (I Q) data.


In [ ]:
 !wget http://opendata.deepsig.io/datasets/2018.01/2018.01.OSC.0001_1024x2M.h5.tar.gz
 !pwd
 !ls
 !tar -xvzf 2018.01.OSC.0001_1024x2M.h5.tar.gz
 !ls
 !rm 2018.01.OSC.0001_1024x2M.h5.tar.gz 

## 2018 Dataset 

### Read in RF Data
3 Arrays will be created. <br>
myData holds the 1024 I and Q time values for each input sample. <br>
myMods holds the one hot encoded RF class for each sample.<br>
mySNRs holds the SNR value for each sample.<br>



In [ ]:
data_file = '/workspace/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
file_handle = h5.File(data_file,'r+')

myData = file_handle['X'][:]  #1024x2 samples 
myMods = file_handle['Y'][:]  #mods 
mySNRs = file_handle['Z'][:]  #snrs  

print(np.shape(myData))
print(np.shape(myMods))
print(np.shape(mySNRs))
file_handle.close()

np.random.seed(0)

### List the SNRs


In [ ]:
snrs = list(np.unique(mySNRs.T[0]))  
print(snrs)

### Define the Modulaton classes

In [ ]:
mods = [
    'OOK',      '4ASK',      '8ASK',      'BPSK',   'QPSK',    '8PSK',
    '16PSK',    '32PSK',     '16APSK',    '32APSK', '64APSK',  '128APSK',
    '16QAM',    '32QAM',     '64QAM',     '128QAM', '256QAM',  
    'AM-SSB-WC','AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK','OQPSK']

num_classes = np.shape(mods)[0]
print("The number of classes is ", num_classes)


### Examine RF input samples
The samples in data set are ordered by class, let's print out one example from each class.

In [ ]:
#turn off warning about more than 10 figures plotted
plt.rcParams.update({'figure.max_open_warning': 0})

def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

size = np.size(myData, axis = 0)
step = size//24

for x in my_range(100000, (size-1), step):
  plt.figure()
  plt.suptitle( mods[np.argmax(myMods[x])])
  plt.plot(myData[x,:,0])
  plt.plot(myData[x,:,1])

### Examine Input Data Range
The input datat is close to being centered around 0, and since the standard deviation is around 1.57, most of the data lies between -6 an +6 with some outliers. 

In [ ]:
print ("Max value of the data set = ", np.max(myData))
print ("Min value of the data set = ", np.min(myData))
print ("Mean value of the data set = ", np.mean(myData))
print ("Standard Deviation of the data set ", np.std(myData) )

Lets examine where the outlyers are comming from. We will see below that all data is between -5 an 5 expect for AM-SSB-WC and the AM-SSB-SC modulations

In [ ]:
length = (myData.shape[0])
limit = 5
Max = np.zeros(length)
Min = np.zeros(length)
for i in range(0,length):
  Max[i] = (np.max(myData[i,:,0]))
  Min[i] = (np.min(myData[i,:,0]))
  if(Max[i] > limit or Min[i] < -limit):
   print ("index =", i, mods[np.argmax(myMods[i])])
plt.figure()
plt.plot(Max)
plt.plot(Min)

### Remove AM-SSB-WC and AM-SSB_SC from the data set
If we leave the AM-SSB-WC and AM-SSB-SC modulations in the data set we will see lower accuracy after quantizing the model to INT8. This is becuase we can more accurately quantize the floating point input data if it is over a smaller range with fewer outlyers as in seen in the other modulations 

In the next step we will remove these two modulations from the data set. If you want to leave these modulations in, you can skip the next step. Leaving these in will cause an additonal 5% accuracy drop after quantizing.

In [ ]:
#Skip this entire panel if you want to leave AM-SSB-WC and AM-SSB-SC modulations in the data set
myData = np.concatenate((myData[0:1810432], myData[2023424:2555904]),axis=0)
mySNRs = np.concatenate((mySNRs[0:1810432], mySNRs[2023424:2555904]),axis=0)
myMods = np.concatenate((myMods[0:1810432], myMods[2023424:2555904]),axis=0)

#re-onehot encode myMods to 22 from 24
length = (np.size(myMods, axis=0))
temp = np.concatenate((myMods[:,0:17],myMods[:,19:24]), axis=1)
myMods = temp

mods = [
    'OOK',      '4ASK',      '8ASK',      'BPSK',   'QPSK',    '8PSK',
    '16PSK',    '32PSK',     '16APSK',    '32APSK', '64APSK',  '128APSK',
    '16QAM',    '32QAM',     '64QAM',     '128QAM', '256QAM',  
    'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK','OQPSK']

num_classes = np.shape(mods)[0]
print("The number of classes is ", num_classes)


print(np.shape(myData))
print(np.shape(mySNRs))
print(np.shape(myMods))

print ("Max value of the data set = ", np.max(myData))
print ("Min value of the data set = ", np.min(myData))
print ("Mean value of the data set = ", np.mean(myData))
print ("Standard Deviation of the data set ", np.std(myData) )


### Now lets look at how the SNRs are distributed across the data set
As you can see each SNR appears an equal number of times across the data set


In [ ]:
plt.figure()
plt.suptitle("SNR Distribution")
plt.hist(mySNRs, bins = [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32]) 
plt.show()

### Reshape RF data to 2D Matrix
We will reshape both the I and Q data from a 1024 long vector to 2D matrix to be conpatabile with 2D convolution commands supported by the DPU. 

In [ ]:
myData = myData.reshape(myData.shape[0], 32, 32, 2) 
# Change to N,C,H,W
myData = np.transpose(myData, (0,3,1,2))

### Slpit Data into Trainnig and Validation set
We will use 80% of the data for the Training set and 20% for the Test set. 
The random_state input to the the train_test_split function is set to 0, which means the 80/20 split will be done in a repeatable manner. The splt is done using the scikir-learn tran_test_split function.

In [ ]:
X_train ,X_test ,Y_train ,Y_test, Z_train, Z_test =train_test_split(myData, myMods, mySNRs, test_size=0.2, random_state=0)

print (np.shape(X_test))
print (np.shape(Y_test))
print (np.shape(Z_test))
print (np.shape(X_train))
print (np.shape(Y_train))
print (np.shape(Z_train))

# remove variables to save memory
del myData, myMods, mySNRs


### Define Dataset Class and loaders

In [ ]:
class RfDataset(Dataset):
    
    def __init__(self, X_data, Y_data):
        self.X_data = X_data
        self.Y_data = Y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.Y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

batch_size = 1024
train_dataset = RfDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train))
test_dataset = RfDataset(torch.from_numpy(X_test).float(), torch.from_numpy(Y_test))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,  shuffle=True)


## Build a Simple CNN  Model 
Here we construct a model with 4 convolutiona layers, followed by 3 fc layers.

In [ ]:
class CNN(nn.Module):
    def __init__(self,p):
        super(CNN, self).__init__()

        self.network = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=(5,5), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=(5,5), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=(5,5), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, kernel_size=(5,5), stride=2,  padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Flatten(),
            nn.Linear((16*8*8), 256),
            nn.Linear(256, 128),
            nn.Linear(128, 22),
            nn.Softmax(dim=1)
            )
    def forward(self, x):
        x = self.network(x)
        return x
    
# model summary
model = CNN(0.5).to(device)
summary (model, (2, 32, 32))

Lets verify that our model is working. 
Because the weight are unitialize, all the probabilites will be close to a random guess of 0.045 (1/22)

In [ ]:
inputs=  torch.from_numpy(X_test[0:1])
print(np.size(X_test[0]))
inputs = inputs.to(device)
predict = model.eval()(inputs)
print(np.size(predict))
print(predict)

In [ ]:
print(inputs.shape)

## TRAIN

### Define Training and Test functions

In [ ]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    '''
    train the model
    '''
    model.train()
    counter = 0
    epoch_loss = 0
 
    print("\nEpoch "+str(epoch), end=" ")
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()
        counter += 1
        if (counter%20 == 1):
             print(end=".")
    epoch_loss += output.shape[0] * loss.item()
    return epoch_loss

def test(model, device, test_loader):
    '''
    test the model
    '''
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            target = target.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    acc = 100. * correct / len(test_loader.dataset)
    return acc


### Start Training

In [ ]:
!mkdir -p checkpoints
nb_epoch = 200 # number of epochs to train on
learnrate = 0.0002
optimizer = optim.Adam(model.parameters(), lr=learnrate)
criterion = nn.CrossEntropyLoss()
best_acc = 0
stop_count = 0
# training with test after each epoch
for epoch in range(1, nb_epoch + 1):
    loss= train(model, device, train_loader, criterion, optimizer, epoch)
    print("\nTraining Loss: ", loss/len(train_loader))
    new_acc = test(model, device, test_loader)
    print("Test Set Accuracy:",format(new_acc), "%")

    if(new_acc <= best_acc):
        stop_count += 1
    else:
        stop_count = 0
        best_acc = new_acc
        # save checkpoint
        torch.save(model.state_dict(),"/workspace/checkpoints/model.pth")
    if(stop_count == 5):
        print("No improvment in accuracy, early stopping")
        break

## Evaluate Model Performance

In [ ]:
#Reload model weights in case it was closed. 
model.load_state_dict(torch.load("/workspace/checkpoints/model.pth"))
score = test(model, device, test_loader)
print("Model Accuracy:",score)


### The Top1 accuracy should be close to 54%

### Generate Classification Report

In [ ]:
batchsize = 1024
total_batches = int(np.shape(X_test)[0]/batchsize) 
y_pred = []
y_actual = []

for i in (range(0, total_batches)):
   x_batch = X_test[i*batchsize:i*batchsize+batchsize]
   inputs =  torch.from_numpy(x_batch)
   inputs = inputs.to(device)
   Y_pred = model.eval()(inputs)
   Y_pred_cpu = Y_pred.cpu().detach().numpy()
   y_pred[i*batchsize:i*batchsize+batchsize] = np.argmax(Y_pred_cpu, axis = 1)
   y_actual[i*batchsize:i*batchsize+batchsize] = np.argmax(Y_test[i*batchsize:i*batchsize+batchsize], axis = 1)
classificationreport_fp = classification_report(y_actual,y_pred, target_names=mods)
print(classificationreport_fp)

Precision Measures the  Accuracy of the positive predictions.
Precision = TP/(TP + FP)

Recall Measures the fraction of positives that were correctly identified.
Recall = TP/(TP+FN)

The F1 score is measure of a weighted harmonic mean of precision and recall. 

Support is the number of values for each class.

Looking at the f1- scores for the different classes, we can see that the model is more accuracte with some classses than others.
For example, the model is not able to correctly identify the higher order PSK and QAM modulations as other classes.   

### Accuracy vs. SNR
Now lets see how the model accuracy is effcted by SNR.

In [ ]:
#Load model in case it was closed
model = CNN(0).to(device)
model.load_state_dict(torch.load('/workspace/checkpoints/model.pth'))
model.eval() 

batchsize = 1024
progress = ProgressBar()
snrlist = np.unique(Z_test)
acc_snr_arr = []

# interate over SNRs
for snr in progress(snrlist):
    acc_arr = []
    i_SNR = np.where(Z_test==snr)
    X_SNR = X_test[i_SNR[0],:,:]
    Y_SNR = Y_test[i_SNR[0],:]
    X_SNR_len = np.shape(X_SNR)[0]
    total_batches = int(X_SNR_len/batchsize)
    
    for i in (range(0, total_batches)):
        x_batch, y_batch = X_SNR[i*batchsize:i*batchsize+batchsize], Y_SNR[i*batchsize:i*batchsize+batchsize]
        
        # model prediction
        inputs=  torch.from_numpy(x_batch)
        inputs = inputs.to(device)
        pred = model(inputs)
        pred = pred.cpu().detach().numpy()
        
        #Pediction values are onehote, corresponding to indices representing different modulation types
        pred_ind = np.argmax(pred, axis=1)
        expected_ind = np.argmax(y_batch, axis=1)
        matches  = sum(np.equal(pred_ind, expected_ind))
        acc      = matches/batchsize
        acc_arr.append(acc)

    # Average the per-batch accuracy values
    accuracy = np.mean(acc_arr)
    acc_snr_arr.append(accuracy)
    print("SNR: ", snr, "accuracy", accuracy)

In [ ]:
plt.figure(figsize=(1,1))
plt.show()
fig= plt.figure(figsize=(10,8))
plt.plot(snrlist, acc_snr_arr, 'bo-', label='accuracy')
plt.ylabel('Accuracy')
plt.xlabel('SNR')
plt.title("Accuracy vs, SNR for Floating Point Model")
plt.legend()
plt.axis([-22, 32, 0, 1.0])
plt.grid()

Are you can see accurccy SNRs below -10db the Top1 accuracy is no better than a random guess (1/24), and once SNR is above 10db the Top1 accuracy is over 80%.

## Vitis AI
The Vitis-AI tools will be used the Quantize and Compile the model for accleration on the DPU. <br> 

## Quantize Model to INT8
The Vitis-AI Quantizer uses a  small set of unlabeled samples to analyze the distribution of the activations. We will use 1000 input samples from the test set, and enable Fast Fine Tuning <br>


In [ ]:
def quantize(device, float_model, quant_mode, batchsize, fast_finetune, deploy, output_dir):
  # load trained model
  model = CNN(0).to(device)
  model.load_state_dict(torch.load(float_model))

  # force to merge BN with CONV for better quantization accuracy
  optimize = 1
  subset_len = 1000

  # override batchsize if in test mode
  if (quant_mode=='test' and deploy):
      if(batchsize != 1):
        print("Forcing batch size to 1")
        batchsize = 1
  
  input = torch.randn([batchsize, 2, 32, 32])
  quantizer = torch_quantizer(quant_mode, model, input, bitwidth=8, output_dir=output_dir) 
  quantized_model = quantizer.quant_model

  calib_dataset = RfDataset(torch.from_numpy(X_test[0:subset_len]), torch.from_numpy(Y_test[0:subset_len]))
  calib_loader = torch.utils.data.DataLoader(calib_dataset, batch_size=batchsize,  shuffle=False)
    
  if fast_finetune == True:  
    if quant_mode == 'calib':
      test(quantized_model, device, calib_loader)  
      quantizer.fast_finetune(test, (quantized_model, device, calib_loader))     
    elif quant_mode == 'test':   
      quantizer.load_ft_param()
   
  # export config
  if quant_mode == 'calib':
    test(quantized_model, device, calib_loader)
    quantizer.export_quant_config()
  # handle quantization result
  elif quant_mode == 'test':
    if(deploy): 
         quantizer.export_xmodel(deploy_check=False, output_dir=output_dir)
    else:
      acc = test(model, device, calib_loader)
      print("Calibration Data Set Accuracy for Floating Point model: ",format(acc))
      acc = test(quantized_model, device, calib_loader)
      print("Calibration Data Set Accuracy for Quantized model: ",format(acc))
     
  return quantized_model

FastTune = True
DeployMode = False 
# generate quantized model
quantize(device, '/workspace/checkpoints/model.pth', 'calib', 10, FastTune, DeployMode, '/workspace/quant_model')
         
# evalute quantized model
quantize(device, '/workspace/checkpoints/model.pth', 'test', 10, FastTune, DeployMode, '/workspace/quant_model')

DeployMode = True
# generate xmodel for compilation
quantized_model = quantize(device, '/workspace/checkpoints/model.pth', 'test', 1, FastTune,  DeployMode, '/workspace/quant_model')

!ls -l /workspace/quant_model/

### Evalute  Model INT8 Performance

In [ ]:
score = test(quantized_model, device, test_loader)
print("Test Set Accuracy of Quantized Model:", score)

The Overall Top-1 score has gone down by about 1.5% due to quantization

### Classification Report for INT8 Model

In [ ]:
batchsize = 1024
total_batches = int(np.shape(X_test)[0]/batchsize) 
y_pred = []
y_actual = []

for i in (range(0, total_batches)):
   x_batch = X_test[i*batchsize:i*batchsize+batchsize]
   inputs =  torch.from_numpy(x_batch)
   inputs = inputs.to(device)
   Y_pred = quantized_model.eval()(inputs)
   Y_pred_cpu = Y_pred.cpu().detach().numpy()
   y_pred[i*batchsize:i*batchsize+batchsize] = np.argmax(Y_pred_cpu, axis = 1)
   y_actual[i*batchsize:i*batchsize+batchsize] = np.argmax(Y_test[i*batchsize:i*batchsize+batchsize], axis = 1)
classificationreport_fp = classification_report(y_actual,y_pred, target_names=mods)
print(classificationreport_fp)

### Accuracy vs SNR for INT8 Model

In [ ]:
batchsize = 1024
progress = ProgressBar()
snrlist = np.unique(Z_test)
acc_snr_arr = []

# interate over SNRs
for snr in progress(snrlist):
    acc_arr = []
    i_SNR = np.where(Z_test==snr)
    X_SNR = X_test[i_SNR[0],:,:]
    Y_SNR = Y_test[i_SNR[0],:]
    X_SNR_len = np.shape(X_SNR)[0]
    total_batches = int(X_SNR_len/batchsize)
    
    for i in (range(0, total_batches)):
        x_batch, y_batch = X_SNR[i*batchsize:i*batchsize+batchsize], Y_SNR[i*batchsize:i*batchsize+batchsize]
        
        # model prediction
        inputs=  torch.from_numpy(x_batch)
        inputs = inputs.to(device)
        pred = quantized_model(inputs)
        pred = pred.cpu().detach().numpy()
        
        #Pediction values are onehote, corresponding to indices representing different modulation types
        pred_ind = np.argmax(pred, axis=1)
        expected_ind = np.argmax(y_batch, axis=1)
        matches  = sum(np.equal(pred_ind, expected_ind))
        acc      = matches/batchsize
        acc_arr.append(acc)

    # Average the per-batch accuracy values
    accuracy = np.mean(acc_arr)
    acc_snr_arr.append(accuracy)
    print("SNR: ", snr, "accuracy", accuracy)

In [ ]:
plt.figure(figsize=(1,1))
plt.show()
fig= plt.figure(figsize=(10,8))
plt.plot(snrlist, acc_snr_arr, 'bo-', label='accuracy')
plt.ylabel('Accuracy')
plt.xlabel('SNR')
plt.title("Accuracy vs, SNR for INT8 Model")
plt.legend()
plt.axis([-22, 32, 0, 1.0])
plt.grid()

The Accuracy vs SNR looks very similar to the floating point model, expect the accuracy is down by about 2% for higher SNRs from the floating point model

## Compile Model for DPU
The Vitis-AI compiler reads in the quantized model and generates an xmodel file which the instruction set for the Xilinx Deep Learning Processor (DPU). The arhictecture option (-a) is used to specify a json file which indicates which hw target the DPU is being compiled for.

In [ ]:
# Select HW Target
#For ZCU104
#!vai_c_xir -x /workspace/quant_model/CNN_int.xmodel -a /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU104/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

#For ZCU102 
#!vai_c_xir -x /workspace/quant_model/CNN_int.xmodel -a /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU102/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

#For Alveo U50
#!vai_c_xir -x /workspace/quant_model/CNN_int.xmodel -a /opt/vitis_ai/compiler/arch/DPUCAHX8H/U50/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

#For Versal VCK190
!vai_c_xir -x /workspace/quant_model/CNN_int.xmodel -a /opt/vitis_ai/compiler/arch/DPUCVDX8G/VCK190/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

## Generate Graph Visualization with xir tool.
You will see a compiler message about the number of  subgraphs:
Total device subgraph number 3, DPU subgraph number 1 <br>
This means that are 3 subgraphs created, 1 for the input layer, 1 for for everything up the softmax layer (which runs on the DPU), and one for the softmax. <br>

The softmax layer can  optionally be acclerated in programmable logic, however in this tutorial we will implement the softmax layer on the CPU.

You can use the the xir command generate a .png file to visulize the graph layers.

In [ ]:
!ls  vai_c_output/rfClassification.xmodel
!xir png vai_c_output/rfClassification.xmodel xmodel.png

### Write out  samples  of Test Data to be used later for HW testing
The python function we will run in the target board will read in these numpy files containing the RF data, class, and SNR.

In [ ]:
np.save('/workspace/rf_input.npy', np.transpose(X_test[0:1000], (0,2,3,1)))
np.save('/workspace/rf_classes.npy', Y_test[0:1000])
np.save('/workspace/rf_snrs.npy', Z_test[0:1000])

In [ ]:
rfIn=np.load('/workspace/rf_input_works.npy')
rfClasses=np.load('/workspace/rf_classes_works.npy')
rfSNRs=np.load('/workspace/rf_snrs_works.npy')

print(np.shape(rfIn[0]))
print(rfIn[0])
print(rfClasses[0])
print(rfSNRs[0])

Now that a dpu xmodel file has been created you are ready to run on target board. You will need to copy the above 3 files, and the xmodel file from the compiler to your target board.

You can close this notebook by entering CtrlC at the console, close the docker container by entering CtrlD, and the proceed with the Tutorial readme instructions.